<a href="https://colab.research.google.com/github/sxyang1206/IBM-University/blob/main/uni_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks'

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (167 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0

In [ ]:
import os
import time

import pandas as pd

import urllib
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By


In [ ]:
def southampton(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    program_name = soup.find('div',{'class': 'block text-black pt-4 pb-10 md:py-4'}).h1.get_text()
    print(program_name)

    module_list = []
    module_card = soup.find_all('div',{'class': 'module-cards'})
    for i in module_card:
        course_list = i.find_all('h3',{'class': 'card-title font-bold text-xl text-endeavour'})
        module_list.extend(course_list)
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def ucl(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    try:
      program_name = soup.find('div',{'class': 'node node-undergraduate-programme-page'}).h1.get_text()
    except:
      program_name = soup.find('div',{'class': 'node node-graduate-taught-programme-page'}).h1.get_text()

    print(program_name)

    module_list = []
    module_card_m = soup.find_all('div',{'class': 'prog-modules-mandatory'})
    module_card_o = soup.find_all('div',{'class': 'prog-modules-optional'})
    for i in module_card_m:
        course_list = i.find_all('a')
        module_list.extend(course_list)
    for i in module_card_o:
        course_list = i.find_all('a')
        module_list.extend(course_list)
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def warwick(url):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(url)
  time.sleep(1)
  soup = BeautifulSoup(driver.page_source)
  driver.close()
  program_name = soup.find('div',{'class': 'id7-page-title'}).h1.get_text().strip()
  print(program_name)

  module_list = []
  module_card_pre = soup.find('div',{'id': 'course-tab-3-collapse'})
  module_card = module_card_pre.find_all('ul')
  for i in module_card:
      course_list = i.find_all('li')
      module_list.extend(course_list)
  module_list = [x.get_text() for x in module_list]
  return program_name, module_list

In [ ]:
from logging import exception
def york(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    try:
        program_name = soup.find('div',{'class': 'c-figure__content c-figure__content--left c-figure__content--half'}).h1.get_text()
    except:
        program_name = soup.find('div',{'class': 'node node-undergraduate-programme-page'}).h1.get_text()

    print(program_name)

    module_list = []
    try:
        module_card_pre = soup.find('div',{'id': 'content_modules'})
        module_card = module_card_pre.find_all('ul',recursive=False)
        for i in module_card:
            course_list = i.find_all('li')
            module_list.extend(course_list)
    except:
        module_card_pre = soup.find_all('div',{'class': 'c-tabs__content'})
        for i in module_card_pre:
            module_card = i.find_all('ul',recursive=False)
            for j in module_card:
                course_list = j.find_all('li')
                module_list.extend(course_list)
    def find_module_name_york(sp):
        try:
            name = sp.a.strong.get_text()
        except:
            try:
                name = sp.strong.a.get_text()
            except:
                name = sp.strong.get_text()
        return name
    module_list = [find_module_name_york(x) for x in module_list]
    return program_name, module_list

In [ ]:
york('https://www.york.ac.uk/study/undergraduate/courses/bsc-data-science/')

BSc (Hons) Data Science


('BSc (Hons) Data Science',
 ['Software 1: Foundations of Programming for Computer Science',
  'Introduction to Probability and Statistics',
  'Foundations and Calculus',
  'Software 2: Object Oriented Data Structures and Algorithms',
  'DATA: Introduction to Data Science',
  'Multivariable Calculus and Matrices',
  'Systems and Devices 2: Operating Systems, Security, and Networking',
  'Engineering 1: Software and Systems Engineering',
  'Probability and Markov Chains',
  'Intelligent Systems: Machine Learning and Optimisation',
  'Linear Algebra',
  'Statistical Inference and Linear Models',
  'Computer Science project',
  'Governance in Data Science',
  'Cloud-Based Data Analysis',
  'Computer Vision and Graphics',
  'Qualitative Approaches to Investigating UX',
  'Autonomous Robotic Systems Engineering',
  'Engineering 2: Automated Software Engineering',
  'Interaction Design and Evaluation',
  'Statistical Pattern Recognition',
  'Generalised Linear Models',
  'Decision Theory and

In [ ]:
def liverpool(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    fl = soup.select('#top > div.faculty-level-container > div')[0].header.h1
    # fl = soup.find('div',{'class': 'faculty-level-inner'}).header.h1
    program_name = fl.span.get_text() + fl.button.get_text()
    print(program_name)

    module_list = []
    module_card = soup.find_all('div',{'class': 'compulsory modules-list'})
    for i in module_card:
        course_list = i.find_all('summary')
        module_list.extend(course_list)
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def lse(url):
    # chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    # chrome_options.add_argument('--no-sandbox')
    # chrome_options.add_argument('--disable-dev-shm-usage')
    # driver = webdriver.Chrome(options=chrome_options)
    # driver.get(url)
    # time.sleep(1)
    # soup = BeautifulSoup(driver.page_source)
    # driver.close()
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    program_name = soup.find('h1',{'class': 'pageTitle'}).get_text()
    print(program_name)

    module_list = soup.find_all('a', href=lambda x: x and x.startswith("https://www.lse.ac.uk/resources/calendar/courseGuides"))
    module_list.extend(soup.find_all('a', href=lambda x: x and x.startswith("http://www.lse.ac.uk/resources/calendar/courseGuides")))
    def find_module_name_lse(sp):
        try:
            name = sp.strong.get_text()
        except:
            name = sp.get_text()
        return name

    module_list = [find_module_name_lse(x) for x in module_list]
    module_list = [module for module in module_list if module != 'course guide']

    return program_name, module_list

In [ ]:
def manchester(url):
    url_new = url + 'all-content/#course-profile'

    r = requests.get(url_new)
    soup = BeautifulSoup(r.text)

    program_name = soup.find('div',{'class': 'heading'}).h1.get_text()
    print(program_name)

    module_list = []
    module_card = soup.find_all('table',{'class': 'course-units'})
    for i in module_card:
        course_list = i.tbody.find_all('a')
        module_list.extend(course_list)
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def newcastle(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    program_name = soup.find('div',{'class': 'nameClass'}).h1.get_text()
    print(program_name)

    module_list = []
    try:
        module_card = soup.find_all('section',{'class': 'modules'})
        for i in module_card:
            module_card_sub = i.find_all('table')
            for j in module_card_sub:
                course_list = j.tbody.find_all('tr')
                module_list.extend(course_list[1:])
    except:
        module_card = soup.find_all('div',{'class': 'responsiveTable0','id':'responsiveTable0'})
        for i in module_card:
            module_card_sub = i.find_all('table')
            for j in module_card_sub:
                course_list = j.tbody.find_all('tr')
                module_list.extend(course_list)
    module_list = [x.td.a.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
newcastle('https://search.ncl.ac.uk/s/redirect?collection=neu-web-courses&url=https%3A%2F%2Fwww.ncl.ac.uk%2Fpostgraduate%2Fdegrees%2F5124f%2F&auth=n9J0qyFR9nrI3JcdzaNH4w&profile=_default&rank=211&query=%7CstencilsCourseType%3Apostgraduate')

Digital Business MSc
[]


('Digital Business MSc', [])

In [ ]:
def birmingham(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    program_name = soup.find('div',{'class': 'hero__inner'}).h1.get_text().strip()
    print(program_name)

    module_list = []
    module_card_pre = soup.find('div',{'id': 'Modules_content'})
    module_card = module_card_pre.find_all('ul')
    for i in module_card:
        course_list = i.find_all('li')
        module_list.extend(course_list)
    def find_module_name_birmingham(sp):
      try:
        module_name = sp.a.get_text()
      except:
        module_name = sp.get_text()[:-12]
      return module_name
    module_list = [find_module_name_birmingham(x) for x in module_list]
    return program_name, module_list

In [ ]:
def bristol(url):

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    course_url = driver.find_element(By.XPATH,"//a[starts-with(@href, 'https://www.bris.ac.uk/unit-programme-catalogue/')]").get_attribute('href')
    program_name = driver.find_element(By.CLASS_NAME,'hero__text').find_element(By.TAG_NAME, 'h1').text
    print(program_name)
    driver.close()

    try:
      course_url = course_url.replace("byCohort=2", "ayrCode=2")
    except:
      course_url = course_url
    r = requests.get(course_url)
    soup = BeautifulSoup(r.text)
    year_pre = soup.select_one('#uobcms-content > div > div > div.column.grid_8 > ul')
    year_list = year_pre.find_all('li')
    module_list = []
    module_card = soup.find_all('table',{'class': 'table-basic'})
    for i in module_card:
        course_list = i.find_all('a')
        module_list.extend(course_list)

    if len(year_list) > 1:
      for i in year_list[1:]:
        url_sub = 'https://www.bris.ac.uk' + i.a.attrs['href']
        r = requests.get(url_sub)
        soup = BeautifulSoup(r.text)
        module_card = soup.find_all('table',{'class': 'table-basic'})
        for j in module_card:
            course_list = j.find_all('a')
            module_list.extend(course_list)
    module_list = [x.get_text() for x in module_list]
    return program_name,module_list


In [ ]:
def cardiff(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    program_name = soup.find('h1',{'id': 'page-title'}).get_text()
    print(program_name)

    module_list = soup.find_all('a',{'class': 'module-information ga-event'})
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def nottingham(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    try:

        program_name = soup.find('div',{'class': 'bannerText'}).h1.get_text()
        print(program_name)

        module_list = soup.find_all('div',{'class': 'cmp-coursemodule__box--row cmp-coursemodule__box--full'})
        module_list = [x.p.get_text().strip() for x in module_list]
    except:
        program_name = soup.find('div',{'id': 'titleKeyFacts'}).h1.get_text()
        print(program_name)
        module_card = soup.find_all('div',{'class': 'moduleBlock'})
        module_list = []
        def find_module_name_nottingham(sp):
          try:
            module_name = sp.get_text().strip().replace(sp.find('span',{'class':"courseCredits"}).text.strip(), "").strip()
          except:
            module_name = sp.get_text().strip()
          return module_name

        for i in module_card:
            course_list = i.find_all('div',{'class': 'js-expandmore'})
            module_list.extend(course_list)
        module_list = [find_module_name_nottingham(x) for x in module_list]
    return program_name, module_list




In [ ]:

def oxford(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    program_name = soup.find('header',{'class': 'main-title'}).h1.get_text()
    print(program_name)

    try:
        module_list = []
        module_card_pre = soup.find('div',{'id': 'content-tab--2'})
        module_card = module_card_pre.find_all('table')

        for i in module_card:
            course_list = i.find('ul').find_all('li',recursive = False)
            for j in course_list:
              course_list2 = j.find_all('li')
              module_list.extend(course_list2)
    except:

        module_list = []
        module_card_pre = soup.find('div',{'id':'content-tab'})
        module_card = module_card_pre.find_all('div')
        for i in module_card:
            course_list = i.find_all('h4')
            module_list.extend(course_list)

    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def queenmary(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    try:
        program_name = soup.find('div',{'class': 'course-title grid__item grid__item--1/2'}).h1.get_text()
        print(program_name)

        module_list = []
        module_card_pre = soup.find('div',{'id': 'structure-tabs'})
        module_list = module_card_pre.find_all('li')
        module_list = [x.get_text().strip() for x in module_list]

    except:
        program_name = soup.find('div',{'class': 'pgt-header__panel'}).h1.get_text()
        print(program_name)

        module_list = []
        module_list = soup.find_all('h4',{'class':'disclosure-box__title'})
        module_list = [x.get_text().strip() for x in module_list]
    return program_name, module_list

In [ ]:
def queenmary_bel(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    try:
        program_name = soup.find('header',{'class': 'cf-banner-new'}).h1.get_text().strip()
    except:
        program_name = soup.find('div',{'class': 'cf-banner-new'}).h1.span.get_text().strip()

    print(program_name)

    module_list = []
    module_card_pre = soup.find('div',{'id': 'modules'})
    module_list = module_card_pre.find_all('div',{'class':'icon-list plus display-block'})
    module_list = [x.a.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def sheffied(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    try:
        program_name = soup.find('div',{'class': 'topdetail'}).h1.get_text().strip()

    except:
        program_name = soup.find('h1',{'class': 'pgtitle'}).get_text().strip()
    print(program_name)

    module_list = []
    module_card = soup.find_all('div',{'class': 'paragraph paragraph--type--ug-course-structure-module-block paragraph--view-mode--full'})
    for i in module_card:
        course_list = i.find_all('dt')
        module_list.extend(course_list)
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def edinburgh(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    program_name = soup.find('h1',{'class': 'page-header'}).get_text().strip()
    try:
      program_lel = soup.find('h1',{'class': 'page-header'}).find('small').get_text().strip()
    except:
      program_lel = ''
    program_name = program_name.replace(program_lel,'').strip() +  program_lel
    print(program_name)

    url_course = soup.find('a', href=lambda x: x and x.startswith("http://www.drps.ed.ac.uk/23-24/")).attrs['href']
    r = requests.get(url_course)
    soup = BeautifulSoup(r.text)


    module_list = []
    module_list = soup.find_all('div',{'class':'dpt-course-card dpt-flex__item dpt-flex'})


    module_list = [x.span.a.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def exeter(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    program_name = soup.find('span',{'id': 'award'}).get_text() + soup.find('span',{'class': 'exeter-course-title'}).get_text().replace(soup.find('span',{'class': 'exeter-course-title'}).span.get_text(),'').strip()

    print(program_name)


    module_list = []
    try:
        course_url = soup.find('iframe',{'class':'rsize'}).attrs['src']
        r = requests.get('https:'+course_url)
        module_card_pre = BeautifulSoup(r.text)

    except:
        module_card_pre = soup.find('div',{'id': 'course-content-accordion'})

    module_card = module_card_pre.find_all('table')
    for i in module_card:
        course_list = i.find_all('tr')[1:]
        module_list.extend(course_list)

    def find_module_name_exeter(sp):
      sp_list = sp.find_all('td')
      if len(sp_list)>1:
        return sp_list[1].get_text()
      else:
        return None

    module_list = [text for text in (find_module_name_exeter(x) for x in module_list) if text is not None]
    return program_name, module_list

In [ ]:
def glasgow(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    program_name = soup.find('div',{'id': 'prog-title'}).h1.get_text().replace(soup.find('div',{'id': 'prog-title'}).h1.span.get_text(),' ').strip() + soup.find('div',{'id': 'prog-title'}).h1.span.get_text()
    print(program_name)

    module_list = []
    module_list = soup.find_all('a', href=lambda x: x and x.startswith("?card=course&"))
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list

In [ ]:
def ic(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    program_name = soup.find('h1',{'class': 'masthead-full-image__card-title'}).get_text()+ soup.find('li',{'class': 'course-tags-list__qualification'}).get_text()
    print(program_name)

    module_list = []
    module_card = soup.find_all('section', {'aria-labelledby':lambda x: x and x.startswith("course-structure-tab-")})
    for i in module_card:
        course_list_1 = i.find_all('span',{'class':'accordion__title'})
        course_list_2 = i.find_all('li')
        module_list.extend(course_list_1)
        module_list.extend(course_list_2)
    module_list = [x.get_text() for x in module_list]
    return program_name, module_list


In [ ]:
def kcl(url):
  return '',[]

In [ ]:
def leeds(url):
  return '',[]

In [ ]:
def cambridge(url):
  return '',[]

In [ ]:
def durham(url):
  return '',[]

**Leeds & Cambridge have no module list detail**

**Durham can not use web scrapping**

**oxford is not commonly used by some subjects**

**kcl left now**



In [ ]:
df_raw = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/module.xlsx')
df_raw.head()

,uni,subject,url,module
0,University of Edinburgh,NaN,https://www.ed.ac.uk/studying/undergraduate/de...,NaN
1,University of Edinburgh,NaN,https://www.ed.ac.uk/studying/undergraduate/de...,NaN
2,University of Edinburgh,NaN,https://www.ed.ac.uk/studying/undergraduate/de...,NaN
3,University of Edinburgh,NaN,https://www.ed.ac.uk/studying/undergraduate/de...,NaN
4,University of Edinburgh,NaN,https://www.ed.ac.uk/studying/undergraduate/de...,NaN


In [ ]:
df_raw['uni'].value_counts()

UNIVERITY OF LIVERPOOL       61
NEWCASTLE UNIVERSITY         45
Uni of Sheffield             41
UNIVERSITY OF MANCHESTER     40
UNIVERSITY OF NOTTINGHAM     35
UNIVERSITY OF LEEDS          32
QUEEN MARY                   28
University of Edinburgh      27
QUEEN MARY BELFAST           25
university of southampton    23
University of Exeter         22
Imperial College London      22
university of york           21
UCL                          19
University of Bristol        13
Kings College London         13
University of Glasgow        13
university of Warwick        12
University of Cardiff        11
University of Birmingham     10
LSE                          10
OXFORD                        7
University of Cambridge       5
University of Durham          4
Name: uni, dtype: int64

In [ ]:
university_functions = {
    'UNIVERITY OF LIVERPOOL': liverpool,
    'NEWCASTLE UNIVERSITY': newcastle,
    'Uni of Sheffield': sheffied,
    'UNIVERSITY OF MANCHESTER': manchester,
    'UNIVERSITY OF NOTTINGHAM': nottingham,
    'QUEEN MARY': queenmary,
    'University of Edinburgh': edinburgh,
    'QUEEN MARY BELFAST': queenmary_bel,
    'university of southampton': southampton,
    'University of Exeter': exeter,
    'Imperial College London': ic,
    'university of york': york,
    'UCL': ucl,
    'University of Bristol': bristol,
    'University of Glasgow': glasgow,
    'university of Warwick': warwick,
    'University of Cardiff': cardiff,
    'University of Birmingham': birmingham,
    'LSE': lse,
    'OXFORD': oxford,
    'University of Cambridge': cambridge,
    'University of Durham': durham,
    'Kings College London': kcl,
    'UNIVERSITY OF LEEDS': leeds
    }

In [ ]:
def write_data(university_name, url, excel_file_path):
    data_fetch_function = university_functions[university_name]
    try:
        subject_name, module_list = data_fetch_function(url)
        modules_str = ', '.join(module_list)

    except:
        subject_name, modules_str = '',''
    try:
        df = pd.read_excel(excel_file_path)
    except:
        df = pd.DataFrame(columns=['University','Subject','Modules','url'])

    new_row = {'University': university_name,'Subject':subject_name, 'Modules': modules_str,'url':url}
    df = df.append(new_row, ignore_index=True)
    df.to_excel(excel_file_path, index=False)
    time.sleep(1)



In [ ]:
import re
def clean_string(input_string):
    cleaned_string = re.sub(r'\s+', ' ', input_string)
    cleaned_string = cleaned_string.strip()
    return cleaned_string

In [ ]:
df_raw_liv = df_raw[df_raw['uni'].str.contains('Birmingham') ]
df_raw_liv

,uni,subject,url,module
97,University of Birmingham,NaN,https://www.birmingham.ac.uk/postgraduate/cour...,NaN
98,University of Birmingham,NaN,https://www.birmingham.ac.uk/postgraduate/cour...,NaN
99,University of Birmingham,NaN,https://www.birmingham.ac.uk/undergraduate/cou...,NaN
100,University of Birmingham,NaN,https://www.birmingham.ac.uk/postgraduate/cour...,NaN
101,University of Birmingham,NaN,https://www.birmingham.ac.uk/postgraduate/cour...,NaN
102,University of Birmingham,NaN,https://www.birmingham.ac.uk/undergraduate/cou...,NaN
103,University of Birmingham,NaN,https://www.birmingham.ac.uk/undergraduate/cou...,NaN
104,University of Birmingham,NaN,https://www.birmingham.ac.uk/postgraduate/cour...,NaN
105,University of Birmingham,NaN,https://www.birmingham.ac.uk/postgraduate/cour...,NaN
106,University of Birmingham,NaN,https://www.birmingham.ac.uk/postgraduate/cour...,NaN


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/module_full.xlsx'
for index,row in df_raw_liv.iterrows():
    cleaned_uni_name = clean_string(row['uni'])
    cleaned_url = row['url'].strip()
    print(cleaned_url)
    write_data(cleaned_uni_name.strip(),cleaned_url,path)

https://www.birmingham.ac.uk/postgraduate/courses/taught/business/business-analytics
MSc Business Analytics


<ipython-input-33-0e07c9ed84ac>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


https://www.birmingham.ac.uk/postgraduate/courses/taught/computer-science/data-science
Data Science Masters/MSc


<ipython-input-33-0e07c9ed84ac>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


https://www.birmingham.ac.uk/undergraduate/courses/computer-science/computer-science
Computer Science BSc


...

<ipython-input-33-0e07c9ed84ac>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


https://www.birmingham.ac.uk/postgraduate/courses/taught/mechanical-engineering/robotics
Robotics Masters/MSc


<ipython-input-33-0e07c9ed84ac>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


https://www.birmingham.ac.uk/postgraduate/courses/taught/computer-science/artificial-intelligence
Artificial Intelligence and Machine Learning Masters/MSc


<ipython-input-33-0e07c9ed84ac>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [ ]:
york('https://www.ucl.ac.uk/prospective-students/undergraduate/degrees/data-science-bsc-2024')

Data Science BSc


('Data Science BSc', [])